# Chapter 4: From neurons to populations

### Load needed modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Section 4.2: Synaptic mechanisms and dendritic processing

### 4.2.3: Modeling synaptic responses
The variation of a membrane potential initiated from a presynaptic spike can be described as a postsynaptic potential (PSP). Excitatory synapses increase a membrane potential in what is called the excitatory postsynaptic potential (EPSP).

#### Membrane potential
In the code below, we simulate the time course of the membrane potential (V) of the postsynaptic neuron in a synapse (equation 4.1)

In [ ]:
# Voltage over time for non-NMDA synapse
t = np.arange(0, 10, 0.1)
v = t*np.exp(-t/2)
plt.plot(t,v); plt.xlabel("t"); plt.ylabel("v")

#### EPSP simulation
In the code above, we simulated the membrane potential (voltage)  **simply as a function of time *(t)***.

We can play with the moment in which the membrane potential peaks and with the amplitude of the current (see eqn 4.1)

In the code below, we go a step further and simulate the excitatory postsynaptic potential with more specific parameters (constants and initial values)

Simulated by shaping the analytical eqn 4.3 into numerical integration (eqn 4.5)

In [ ]:
# Synaptic conductance model to simulate an EPSP

# Setting some constants (won't change) and initial values (will be updated at every integration step)
c_m=1; V_l=0; V_s=10; tau=1; dt=0.1; #c_: capacitance; g_: conductance; v_: voltage; I_: current
g_l=1; g_s=[0]; # _m: membrane; _l: leakage channel; _s: synaptic ion channel
I_l = [0]; I_s = [0];
v_m = [0]; t =  [0];

# Numerical integration using Euler scheme
for step in range(1, int(10/dt)):
    
    # record the time (in ms) in slot step of vector t
    t.append(t[step-1]+dt)
    
    # simulate opening synaptic channels around t = 1 ms
    if abs(t[step]-1) < 0.0001: g_s[step-1] = 1; # an implementation of the delta function (chapter 3)
    
    # calculate the currents at this time
    I_l.append(g_l * (v_m[step-1]-V_l))
    I_s.append(g_s[step-1] * (v_m[step-1]-V_s))
    
    # update conductance and membrane potential
    g_s.append(g_s[step-1]-dt/tau * g_s[step-1])
    v_m.append(v_m[step-1]-dt/c_m*(I_l[step]+I_s[step]))
    
plt.plot(t, v_m, 'k');

**In class** 

Let's try and plot the temporal development of other variables (e.g., current of the leakage channel, or conductance of the synaptic channel)

## Section 4.3: The generation of action potentials: Hodgkin-Huxley 

### Hodgkin-Huxley model

The Hodgkin-Huxley is a classical minimal model of action potential generation based on the observations directly recorded from the axons of squid.

This model also works well at approximating the action potential of pyramidal cells in vertebrates such as ourselves.

Although the functions and their python implementation might look complex, they are just the way to fit the experimental observations of Hodgkin and Huxley. For example:
- Some constants: 
    - conductances of Na, K, and leak channels: **g** array.
    - Equilibrium potentials of Na, K, and leak: **V** array.
- Some variables:
    - gating variable for the different ion channels: **x** array
- Some functions, the alpha and beta functions describe the voltage-dependent probability of the activation (n, m) or inactivation (h) channels being open

In summary, this model describes how membrane voltage (v) evolves based on the dynamics of sodium, potassium, and leak currents.

In [ ]:
# Hodgkin-Huxley model
# Max conductances (in units of mS/cm^2) ; 0=K, 1=Na, 2=R
g = np.array([36,120,0.3])
# Battery voltage (in mV); 0=n, 1=m, 2=h
V = np.array([-12,115,10.613])
# Initialization of some variables
v=0; x= np.array([0,0,1]); t=0; dt=0.01
t_rec=[]; v_rec=[]; x_0_rec=[]
# Integration with Euler method
for step in range(1, int(100/dt)):
    t=t+dt
    if t>30 and t<89: I_ext=10
    else: I_ext=0
    # alpha functions used by Hodgkin and Huxley (eqn 4.11 - 4.16)
    Alpha = np.array([0.01*(-v+10)/(np.exp((-v+10)/10)-1),
    0.1*(-v+25)/(np.exp((-v+25)/10)-1),
    0.07*np.exp(-v/20)])
    # beta functions used by Hodgkin and Huxley
    Beta = np.array([0.125*np.exp(-v/80),
                    4*np.exp(-v/18),
                    1/(np.exp((-v+30)/10)+1)])
    # update x = {n,m,h} variables
    x = x+dt*(Alpha*(1-x)-Beta*x)
    # calculate actual conductances g with given n, m, and h
    gx = np.array([g[0]*x[0]**4,g[1]*x[1]**3*x[2],g[2]])
    # Ohm's law
    I = gx*(v-V)
    # update voltage of membrane
    v = v+dt*(I_ext-sum(I))
    # record some variables for plotting after equil.
    t_rec.append(t)
    v_rec.append(v)

# Plotting results
plt.plot(t_rec, v_rec);
plt.xlabel('Time'); plt.ylabel('Voltage')
plt.rcParams.update({'font.size':15})
plt.legend(loc = "center left")

Let's modify some values in the model!

### Activation function of Hodgkin-Huxley neuron
The activation function lets us explore two relevant aspects of the Hodgkin–Huxley neuron.

First, in the basic model, shown with the solid line in the graph, onset of firing follows a sharp threshold (around 6mA/cm2).

Second, there is a fairly limited range of frequencies when the axon fires, starting at about 50 Hz and increasing only slightly with increasing external current. This is interesting because it shows that not all neuronal frequencies are equally likely.

The figure poduced with the code below igure shows a spike count for a given window (currently 1000 ms). That's why it might show more continuous or stepwise increases in spike frequency depending on the size of the time window (more time, more counts and continuity). 

In [ ]:
def count_spikes(v, threshold=0):
    """Count the number of spikes in the membrane potential."""
    spikes = (v[:-1] < threshold) & (v[1:] >= threshold)
    return np.sum(spikes)

# Constants
g = np.array([36, 120, 0.3])
V = np.array([-12, 115, 10.613])
dt = 0.01
simulation_time = 1000  # ms
num_steps = int(simulation_time / dt)

# Range of external currents to test
I_ext_range = np.linspace(0, 20, 50)
firing_rates = []

for I_ext in I_ext_range:
    v = 0
    x = np.array([0, 0, 1])
    v_rec = []

    for step in range(num_steps):
        t = step * dt

        # Alpha functions
        Alpha = np.array([
            0.01 * (-v + 10) / (np.exp((-v + 10) / 10) - 1),
            0.1 * (-v + 25) / (np.exp((-v + 25) / 10) - 1),
            0.07 * np.exp(-v / 20)
        ])

        # Beta functions
        Beta = np.array([
            0.125 * np.exp(-v / 80),
            4 * np.exp(-v / 18),
            1 / (np.exp((-v + 30) / 10) + 1)
        ])

        # Update gating variables (n, m, h)
        x = x + dt * (Alpha * (1 - x) - Beta * x)

        # Conductances
        gx = np.array([g[0] * x[0]**4, g[1] * x[1]**3 * x[2], g[2]])

        # Currents and voltage update
        I = gx * (v - V)
        v = v + dt * (I_ext - sum(I))

        # Record membrane potential
        v_rec.append(v)

    # Calculate firing rate (spikes per second)
    spikes = count_spikes(np.array(v_rec))
    firing_rate = spikes / (simulation_time / 1000)  # spikes per second
    firing_rates.append(firing_rate)

# Plot the activation function
plt.figure(figsize=(10, 6))
plt.plot(I_ext_range, firing_rates, label="Firing Rate")
plt.title("Activation Function: Firing Rate vs. External Current")
plt.xlabel("External Current (\u03bcA/cm^2)")
plt.ylabel("Firing Rate (Hz)")
plt.grid()
plt.legend()
plt.show()

### The Wilson model
The Wilson model is achieved by simplifying the 4 differential equations of Hodgkin-Huxley neuron to 2 equations, and adding two additional differential equations that are essential for simulating the diverse firing patterns of neocortical mammalian neurons.

With it, we can  approximate mammalian spike characteristics in great detail. This includes the shape of single spikes, as well as all major classes of spike characteristics, such as regular spiking neurons (RS), fast spiking neurons (FS), continuously spiking neurons (CS), and intrinsic bursting neurons (IB), by choosing appropriate values of the remaining constants.

In [ ]:
# Wilson neuron
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def ydot(y, t, Iext, g, E, tau):
    
    V=y[0]; R=y[1]; T=y[2]; H=y[3]
    
    gNa = 17.8 + 47.6*V + 33.8*V**2
    R0 = 1.24 +  3.7*V +  3.2*V**2
    T0 = 4.205 + 11.6*V + 8*V**2
    X = np.array([R, gNa, T, H])
    
    Vdot = -1/tau[0]*(g*X@(V-E)-Iext)
    Rdot = -1/tau[1]*(R-R0)
    Tdot = -1/tau[2]*(T-T0)
    Hdot = -1/tau[3]*(H-3*T)
    
    return np.array([Vdot, Rdot, Tdot, Hdot])

# parameters of the model: 0=K 1=Na 2=T 3=H
g = np.array([26, 1, 2.25, 9.5])
E = np.array([-0.95, 0.50, 1.20, -0.95])
tau = np.array([1, 4.2, 14, 45])

# 1: Equilibration: no external input
Iext = 0; y0=np.zeros(4); y0[0]=-1
t=np.arange(0,100,0.1)
y = odeint(ydot, y0, t, args=(Iext, g, E, tau))

# 2: Integration with exteral input
Iext = 1; y0=y[-1,:]
t = np.arange(0,200, 0.1)
y = odeint(ydot, y0, t, args=(Iext, g, E, tau))

plt.plot(t,y[:,0])
plt.xlabel('Time'); plt.ylabel('Membrane potential')

### 4.4. The FitzHugh-Nagumo model
Similarly to the Wilson model, the FN model simplifies Hodgkin-Huxley equations. It doesn't capture as well the biological reality of an action potential as the Hodgkin-Huxley model.

But, by reducing the numer of equations to two (for a voltage activator and a recovery variable), it is able to qualitatively reproduce the action potential generation of neurons, enabling the study of their dynamical features.


In [ ]:
# FitzHugh-Nagumo model
import matplotlib.pyplot as plt

# Parameters and initialization
Iext=0.5; v=[0]; w=[0]; t=[0]; dt=0.1

for step in range(1, int(200/dt)):
    # record time steps
    t.append(t[step-1]+dt)
    # numerical integration and recording
    v.append(v[step-1]+dt*(v[step-1]-v[step-1]**3/3-w[step-1]+Iext)) # voltage
    w.append(w[step-1]+dt*0.08*(v[step-1]+0.7-0.8*w[step-1])) # recovery

plt.plot(t,v); plt.xlabel('t'); plt.ylabel('v')

## Homework
Due Monday, Feb. 10, 13.00. Send to jperez@bcbl.eu in either .doc, .txt, or just by inserting it in the markdown cell below and sending me a copy of this notebook
1. What possible cognitive/language/behavioral mechanism would you be able to simulate with a single neuron model? Feel free to speculate even if the model already exists or you think it might be difficult to implement or biologically implausbile ;)
2. Why the code for simulating the postsynaptic membrane potential is an useful approximation to AMPA and GABA neurons but not NMDAR neurons?
3. If the Hodgkin-Huxley model describes closely the biological reality of action potential generation, why are models as Wilson's or Fitz-Nagumo's useful? Speculate a bit on the application we could make of Hodgkin-Huxley, Wilson, and Fitz-Nagumo models.